<a href="https://colab.research.google.com/github/p82maavd/IAA/blob/main/Practica_2_IAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic Dataset**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install turicreate

Mounted at /content/drive
     |████████████████████████████████| 92.0 MB 14 kB/s 
     |████████████████████████████████| 3.6 MB 26.5 MB/s 
     |████████████████████████████████| 322 kB 42.6 MB/s 
     |████████████████████████████████| 86.4 MB 73 kB/s 
     |████████████████████████████████| 3.5 MB 29.5 MB/s 
     |████████████████████████████████| 18.3 MB 457 kB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 3.8 MB 30.8 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 2.9 MB 38.0 MB/s 
     |████████████████████████████████| 449 kB 54.5 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=63a3822b90e10bb613aa1eeecf244def983f71358fe14b2ba56ef5fba2e9efbd
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics



feature1=[1,1,2,2,3,3,4]
feature2=[2,3,3,4,2,5,1]
label=[1.03,-1.44,4.53, 2.24, 13.27, 5.62, 21.53]
data= [feature1,feature2,label]
dataset = pd.DataFrame (data).transpose()
dataset.columns = ['feature1', 'feature2', 'label']

modelSquare=[]

for x in range(len(dataset.columns)):
  modelSquare.append(random.randint(-5, 5))
modelAbsolute=modelSquare.copy()


def evaluate_model(model,point):
  resultado=0.0
  
  for i in range(len(point)-1):
    resultado+=point[i]*model[i]
  return resultado+model[-1]

def errorFunction(model,df):
  error=0.0
  error_cuadrado=0.0
  for i in range(df.shape[0]):
    punto=df.iloc[i].to_numpy()
    error+=evaluate_model(model,punto)
    error_cuadrado=evaluate_model(model,punto)*evaluate_model(model,punto)
  print("El error absoluto medio es: ", error/df.shape[0])
  print("El error cuadratico medio es: ", error_cuadrado/df.shape[0])

def multipleLinearRegresionSquareTrick(model, point, learning_rate):
  
  new_b=model[-1]+learning_rate*(point[-1]-evaluate_model(model,point))
  model[-1]=new_b
  for i in range(len(dataset.columns)-1):
    new_w=model[i]+learning_rate*point[i]*(point[-1]-evaluate_model(model,point))
    model[i]=new_w

def multipleLinearRegresionAbsoluteTrick(model, point, learning_rate):

  if point[-1]>evaluate_model(model,point):

    new_b=model[-1]+learning_rate
    for i in range(len(dataset.columns)-1):
      new_w=model[i]+learning_rate*point[i]
      model[i]=new_w

  else:

    new_b=model[-1]-learning_rate
    for i in range(len(point)-1):
      new_w=model[i]-learning_rate*point[i]
      model[i]=new_w

  model[-1]=new_b
 
for i in range(0,1000):
  punto=dataset.iloc[random.randint(0,dataset.shape[0]-1)].to_numpy()
  multipleLinearRegresionSquareTrick(modelSquare,punto,0.05)
  multipleLinearRegresionAbsoluteTrick(modelAbsolute,punto,0.05)

print("Modelo de la implementación Square trick: ",modelSquare)
errorFunction(modelSquare,dataset)
print("")
print("Modelo de la implementación Absolute trick: ",modelAbsolute)
errorFunction(modelAbsolute,dataset)


#Turi Create

import turicreate as tc

# Load the data
data =  tc.SFrame(dataset)

# Make a train-test split
#train_data, test_data = data.random_split(0.8)

# Automatically picks the right model based on your data.
model = tc.linear_regression.create(data, target='label',
                                    features = ['feature1', 'feature2'], verbose=False)

print(model.coefficients)
# Save predictions to an SArray
#predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
#results = model.evaluate(test_data)

#print(results)

#Scikit Learn

X= dataset.drop(['label'],axis=1)
y = dataset['label']

#X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2,random_state=1)


regr = linear_model.LinearRegression()
regr.fit(X, y)

print("")
print("Modelo SKlearn: ", regr.coef_, regr.intercept_)

#y_pred=regr.predict(X_test)
#print("Linear Regression Accuracy for Scikit-Learn: %f" % ( metrics.accuracy_score(y_test, y_pred)))



Modelo de la implementación Square trick:  [6.02415679668034, -2.503454759285354, 0.04187502108118675]
El error absoluto medio es:  6.658648386963809
El error cuadratico medio es:  66.86789687137004

Modelo de la implementación Absolute trick:  [6.200000000000008, -2.6999999999999917, 0.10000000000000032]
El error absoluto medio es:  6.557142857142899
El error cuadratico medio es:  70.40571428571455
+-------------+-------+---------------------+---------------------+
|     name    | index |        value        |        stderr       |
+-------------+-------+---------------------+---------------------+
| (intercept) |  None | 0.05240794104558599 |  0.193992579593948  |
|   feature1  |  None |  5.975723751721802  | 0.05204271620406507 |
|   feature2  |  None |  -2.459921780743396 | 0.04304818354787499 |
+-------------+-------+---------------------+---------------------+
[3 rows x 4 columns]


Modelo SKlearn:  [ 6.02426471 -2.47485294] -0.01588235294117979


In [ ]:
#Si queremos intentar probar nuestra implementacion con el dataset complejo, da error de overflow

#dataset = pd.read_csv("/content/drive/MyDrive/Uni/IAA/Hyderabad.csv",header=0)
#dataset=dataset.drop(['Location'],axis=1)
#dfaux= dataset["Price"]
#dataset=dataset.drop(['Price'],axis=1)
#dataset.insert(len(dataset.columns), "Price", dfaux)